# **Homogeneous Coordinates**

In this section we be able to implement the pointconversion(i.e., cartesian coordinate system to homogeneous coordinate system) and geometric transformation like rotation, scaling and translation using Numpy. Furthermore, we will be able to recognize the advantages of the homogeneous system in performing composite transformation.



* [Point Conversion](#point_conversion)

* [Rotation](#rotation)

* [Scaling](#scaling)

* [Translation](#translation)

* [Composite Transformation](#composite_transformation)
    

# **Overview**

Homogeneous coordinates are the coordinates system totally different from the normal coordinate system. In here, we can represent the 2D with 3D homogeneous coordinates, and 3D with 4D homogeneous coordinates. Here, it simply add another component to original coorinate system.

Suppose, the cartesian 2D point is $(x,y)$. This cartesian 2D point you can represent into 3D homogeneous form by introducing a third component to it Z. That is, now we can represent $(x,y)$ with $(xZ, yZ, Z)$.

This homogeneous coordinate system are good as it provides a lot advantages to you like handling of points at infinity, making formulas simpler, making complex transformation processes simpler and so on. It has a range of applications, including computer gaphics and 3D computer vision, where they allow affine transformatons, and projective transformations to be easily represented by a matrix.

Here, we will implement geometric transforamtions like `Rotation`, `Scaling` and `Translation` using homogeneous matrix and also composite transfomation using the above geometric trnasformations.

## **Point Conversion** <a name='point_conversion'></a>

The first excercise is the point conversion where we will perform simple conversion of cartesian coordinates to homogeneous ones and vice-versa. 


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry.polygon import Polygon
from descartes import PolygonPatch

In the homogeneous function, we just add another term, $w$, to the orginal co-ordinate system, $(x,y)$, as shown in the examples below.

$$
(x,y) ---> (x,y,1)\\
(x/w, y/w) ---> (x,y,w)
$$

Now, we need to create a function for the conversion of homogeneous coordinates to cartesian form and cartesian form to homogeneous ones.

Create function `cartesian_to_homogenous()`. Here, we need to just add extra element to original coordinate system.**

Hint: 

(5,5) is cartesian, homogeneous coordinates will be (5,5,1).

In [6]:
def cartesian_to_homogeneous(coordinates):

    # Converting the input to a NumPy array for easier manipulation
    coordinates = np.array(coordinates)
    
    # Adding the homogeneous coordinate (1) as the last element
    homogeneous_coords = np.append(coordinates, 1)
    
    return homogeneous_coords

In [7]:
# Example usage
cartesian_2d = [5, 5]
cartesian_3d = [5, 5, 7]

print("2D Cartesian to Homogeneous:", cartesian_to_homogeneous(cartesian_2d))  

print("3D Cartesian to Homogeneous:", cartesian_to_homogeneous(cartesian_3d))  

2D Cartesian to Homogeneous: [5 5 1]
3D Cartesian to Homogeneous: [5 5 7 1]


#### **Task 2:** 
Create function `homogeneous_to_cartesian()`. Here, we need to just remove the extra element from the given homogeneous coordinate system.

Hint: 

(10, 10, 2) is homogenous coordinates, cartesian coordinates will be (5,5).

In [8]:
def homogeneous_to_cartesian(coordinates):
    '''
    This function converts the homogeneous coordinates to cartesian form.
    
    Parameters
    ----------
    coordinates: list or array, can be 3D, 4D homogeneous coordinates
    
    return
    ------
    cartesian_coords: np.array of cartesian coordinates
    '''
    # Converting the input to a NumPy array for easier manipulation
    coordinates = np.array(coordinates)
    
    # The last element is the scaling factor
    scaling_factor = coordinates[-1]
    
    # Dividing all other elements by the scaling factor to get cartesian coordinates
    cartesian_coords = coordinates[:-1] / scaling_factor
    
    return cartesian_coords


To implement the homogeneous_to_cartesian() function, we need to remove the last element from the homogeneous coordinates and divide the remaining elements by the last element. This is necessary because homogeneous coordinates are scaled by the last element, and to get back to Cartesian form, we need to divide the other components by this scaling factor.

In [11]:
# Example usage
homogeneous_2d = [10, 10, 2]
homogeneous_3d = [15, 30, 45, 3]

print("Homogeneous to Cartesian (2D):", homogeneous_to_cartesian(homogeneous_2d))  

print("Homogeneous to Cartesian (3D):", homogeneous_to_cartesian(homogeneous_3d))  

Homogeneous to Cartesian (2D): [5. 5.]
Homogeneous to Cartesian (3D): [ 5. 10. 15.]


**Input**: The function takes a list or array of homogeneous coordinates (3D or 4D).<br/>
**Conversion**: It divides the first two or three elements (depending on dimensionality) by the last element (the scaling factor).<br/>
**Output**: It returns a NumPy array with the Cartesian coordinates.

## **Rotation** <a name="rotation"></a>

Rotation means turning an object around a certain fixed point, called the center of rotation(COR). In rotation, we rotate or turn an object with certain angle, called the angle of rotation(AOR), $\theta$. 

In rotation, if angle of rotation is positive, it is called anticlockwise rotation, and if the angle of rotation is negative, it is called clockwise rotation. 



In this exercise, we create function to rotate the different 2D shapes(like trianlge, square).

#### **Task 1:** <a name="exercise-2-task-1"></a>
In this task, we create a function to calculate the rotation matrix. 

___Hint:___

   * Use [np.deg2rad()](https://numpy.org/doc/stable/reference/generated/numpy.deg2rad.html) or [np.radians()](https://numpy.org/doc/stable/reference/generated/numpy.radians.html) to convert angle from degree to radians.
   
   * Convert the following 2D rotation matrices into homogeneous form:

   Anticlockwise Rotation matrix:
$$ 
\begin{bmatrix}
cos\theta & -sin\theta
 \\
sin\theta & cos\theta
\end{bmatrix}
{-------->}^{Homogeneous form}
\begin{bmatrix}
cos\theta & -sin\theta & 0
 \\
sin\theta & cos\theta & 0
\\ 0 & 0 & 1
\end{bmatrix}
$$
Clockwise Rotation  matrix:
$$ 
\begin{bmatrix}
cos\theta & sin\theta
 \\
-sin\theta & cos\theta
\end{bmatrix}
{-------->}^{Homogeneous form}
\begin{bmatrix}
cos\theta & -sin\theta & 0
 \\
sin\theta & cos\theta & 0
\\ 0 & 0 & 1
\end{bmatrix}
$$

`NOTE: np.cos(np.radians(90)) !=0 or np.cos(np.deg2rad(90)) !=0(Use some logic for this case)`

To implement the get_rotation_matrix() function, we need to compute the 2D rotation matrix in homogeneous form, based on whether the rotation is clockwise or anticlockwise.

The formula for the rotation matrix in homogeneous coordinates is:

Anticlockwise Rotation Matrix (for positive angles):

[
cos
⁡
𝜃
−
sin
⁡
𝜃
0
sin
⁡
𝜃
cos
⁡
𝜃
0
0
0
1
]
​
  
cosθ
sinθ
0
​
  
−sinθ
cosθ
0
​
  
0
0
1
​
  
​
 
Clockwise Rotation Matrix (for negative angles):

[
cos
⁡
𝜃
sin
⁡
𝜃
0
−
sin
⁡
𝜃
cos
⁡
𝜃
0
0
0
1
]
​
  
cosθ
−sinθ
0
​
  
sinθ
cosθ
0
​
  
0
0
1
​
  
​
 
We'll convert the input angle from degrees to radians using np.deg2rad() or np.radians(), then calculate the cosine and sine values, and finally construct the 3x3 homogeneous rotation matrix.

In [13]:
def get_rotation_matrix(angle):
    '''
    This function provides the rotation matrix either for clockwise or anticlockwise rotation.
    
    Parameters
    ----------
        angle: angle of rotation in degrees (positive for anticlockwise, negative for clockwise)
    
    Returns
    -------
        rotation_matrix : 3x3 numpy array in homogeneous form
    '''
    
    # Converting angle to radians
    theta = np.deg2rad(angle)
    
    # Computing cos and sin of the angle
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    # Checking for very small values close to zero due to floating-point precision issues
    if abs(cos_theta) < 1e-10:
        cos_theta = 0
    if abs(sin_theta) < 1e-10:
        sin_theta = 0
    
    # Creating the 3x3 homogeneous rotation matrix
    rotation_matrix = np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta, cos_theta, 0],
        [0, 0, 1]
    ])
    
    return rotation_matrix

In [14]:
# Example usage
angle_anticlockwise = 90  
angle_clockwise = -90     

print("Anticlockwise Rotation Matrix (90°):\n", get_rotation_matrix(angle_anticlockwise))
print("\nClockwise Rotation Matrix (-90°):\n", get_rotation_matrix(angle_clockwise))

Anticlockwise Rotation Matrix (90°):
 [[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]

Clockwise Rotation Matrix (-90°):
 [[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]


#### **Task 2:** <a name="exercise-2-task-2"></a>
In this task, we create a function to perform the action of rotation. 

***Step 1:***  Convert the given coordinates points to homogeneous coordinate system

   * Use `np.array()`, `np.asarray()` for converting list into numpy array
   * Substract given coordinates point from centre of rotation
   * Use `cartesian_to_homogeneous()`, to convert given points ot homogeneous form 
   * Use list comprehension to simplify your steps or can perform multiple logical operations
       
***Step 2:*** Calculate the rotation matric
    
   * Use `get_rotation_matrix()`, for calculation of the rotation matrix
   * Use the center of rotation as origin

***Step 3:*** Calculate the rotated points

   * Use `np.matmul()`, to multiply homogeneous_points and rotating_matrix for calculating the rotated points
   
***Step 4:*** Convert homogeneous rotated points to cartesian form

   * Use `homogeneous_to_cartesian()`


In [21]:
def cartesian_to_homogeneous(coordinates):
    ''' Convert Cartesian coordinates to Homogeneous form '''
    return np.append(np.array(coordinates), 1)

def homogeneous_to_cartesian(coordinates):
    ''' Convert Homogeneous coordinates to Cartesian form '''
    return np.array(coordinates[:-1]) / coordinates[-1]

def get_rotation_matrix(angle):
    ''' Calculate the 2D homogeneous rotation matrix '''
    theta = np.deg2rad(angle)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    # Homogeneous rotation matrix
    rotation_matrix = np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta, cos_theta, 0],
        [0, 0, 1]
    ])
    
    return rotation_matrix

To implement the rotation() function, we need to follow the steps described:

1. Convert the points to homogeneous coordinates.

2. Subtract the points from the center of rotation (COR) to ensure the points are rotated around the COR.
Convert the points to homogeneous form using cartesian_to_homogeneous().
Calculate the rotation matrix.

3. Use the get_rotation_matrix() function to get the rotation matrix based on the angle of rotation.
Multiply the points by the rotation matrix.

4. Use np.matmul() to perform matrix multiplication between the homogeneous points and the rotation matrix.
Convert the rotated points back to Cartesian coordinates.

5. Use homogeneous_to_cartesian() to convert the rotated points back to Cartesian form.

In [22]:
def rotation(points, angle, COR=[0, 0], homogeneous=False):
    '''
    This function performs rotation and returns the rotated points.
    
    Parameters
    -----------
        points: list of x and y coordinates that needs to be rotated
        angle: rotating angle in degrees
        COR: list of x and y coordinates representing the center of rotation
    
    Returns
    -------
        rotated_points: numpy array of rotated points in homogeneous form
        cart_rotated_points: numpy array of rotated points in Cartesian form
    '''
    # STEP 1: Converting points to homogeneous coordinates
    # Translating the points relative to the COR (Center of Rotation)
    translated_points = [np.array(point) - np.array(COR) for point in points]
    
    # Converting to homogeneous coordinates
    homogenous_points = [cartesian_to_homogeneous(point) for point in translated_points]
    
    # STEP 2: Getting the rotation matrix
    rotation_matrix = get_rotation_matrix(angle)
    
    # STEP 3: Rotating points by multiplying with rotation matrix
    rotated_points = [np.matmul(rotation_matrix, point) for point in homogenous_points]
    
    # STEP 4: Converting rotated points back to Cartesian form
    cart_rotated_points = [homogeneous_to_cartesian(point) + np.array(COR) for point in rotated_points]
    
    # Always return both rotated points in homogeneous and cartesian form
    return np.array(rotated_points), np.array(cart_rotated_points)

# Explanation: <br/>
Step 1: The points are first translated relative to the center of rotation (COR), then converted to homogeneous coordinates using the cartesian_to_homogeneous() function.  <br/>
Step 2: The rotation matrix is calculated using the get_rotation_matrix() function.<br/>
Step 3: The homogeneous points are rotated by multiplying with the rotation matrix.<br/>
Step 4: The rotated points are converted back to Cartesian coordinates using the homogeneous_to_cartesian() function, and then translated back to the original position relative to the COR.

In [23]:
# Example usage
points = [[1, 0], [0, 1], [1, 1]]  # Points of a triangle
angle = 90  # Rotate by 90 degrees
COR = [0, 0]  # Rotate around the origin

rotated_points, cart_rotated_points = rotation(points, angle, COR)

print("Homogeneous Rotated Points:\n", rotated_points)
print("\nCartesian Rotated Points:\n", cart_rotated_points)


Homogeneous Rotated Points:
 [[ 6.123234e-17  1.000000e+00  1.000000e+00]
 [-1.000000e+00  6.123234e-17  1.000000e+00]
 [-1.000000e+00  1.000000e+00  1.000000e+00]]

Cartesian Rotated Points:
 [[ 6.123234e-17  1.000000e+00]
 [-1.000000e+00  6.123234e-17]
 [-1.000000e+00  1.000000e+00]]
